In [ ]:
pip install pandas_profiling

In [ ]:
pip install unidecode

In [225]:
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import unidecode
import seaborn as sns
import requests
import zipfile
import io
import os
import glob
import warnings
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

pd.set_option('display.max_rows', None)  # Mostra todas as linhas do DataFrame
pd.set_option('display.max_columns', None)  # Mostra todas as colunas do DataFrame

In [107]:
def generate_url(base_url, year, additional_info=""):
    return f'{base_url}_{year}{additional_info}.zip'

def create_urls(base_url, years, additional_info=""):
    return {year: generate_url(base_url, year, additional_info) for year in years}

# URLs dos arquivos de votação por seção
votacao_base_url = 'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao'
votacao_years = [2020, 2016, 2012, 2008, 2004, 2000]
votacao_urls = create_urls(votacao_base_url, votacao_years, "_PR")

# URLs dos arquivos de bens do candidato (2008 o registro mais recente)
bem_candidato_base_url = 'https://cdn.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato'
bem_candidato_years = [2020, 2016, 2012, 2008]
bem_candidato_urls = create_urls(bem_candidato_base_url, bem_candidato_years)

# URLs dos arquivos de consulta de candidatos
consulta_cand_base_url = 'https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand'
consulta_cand_years = [2020, 2016, 2012, 2008, 2004, 2000]
consulta_cand_urls = create_urls(consulta_cand_base_url, consulta_cand_years)

print("URLs de votação por seção:")
for year, url in votacao_urls.items():
    print(f'{year}: {url}')

print("\nURLs de bens do candidato:")
for year, url in bem_candidato_urls.items():
    print(f'{year}: {url}')

print("\nURLs de consulta de candidatos:")
for year, url in consulta_cand_urls.items():
    print(f'{year}: {url}')

URLs de votação por seção:
2020: https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2020_PR.zip
2016: https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2016_PR.zip
2012: https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2012_PR.zip
2008: https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2008_PR.zip
2004: https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2004_PR.zip
2000: https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2000_PR.zip

URLs de bens do candidato:
2020: https://cdn.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2020.zip
2016: https://cdn.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2016.zip
2012: https://cdn.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2012.zip
2008: https://cdn.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2008.zip

URLs de consulta de candid

In [7]:
# Pasta de destino
output_folder = 'extracao'

# Obter o caminho absoluto para a pasta de destino
absolute_output_folder = os.path.join(os.getcwd(), output_folder)

# Criar a pasta se não existir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Função para baixar e extrair os arquivos
def download_and_extract(url, output_folder, filter_extension=None):
    response = requests.get(url)

    # Verificar se a requisição foi bem-sucedida
    if response.status_code == 200:
        try:
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                # Iterar sobre os arquivos no ZIP
                for file in z.namelist():
                    file_path = os.path.join(output_folder, file)

                    # Verificar a extensão do arquivo se um filtro for fornecido
                    if filter_extension and not file.endswith(filter_extension):
                        continue

                    # Verificar se o arquivo já existe na pasta de destino
                    if not os.path.exists(file_path):
                        # Extrair o arquivo mantendo seu nome original
                        with z.open(file) as source, open(file_path, 'wb') as dest:
                            dest.write(source.read())
                        print(f"Arquivo {file} extraído para {output_folder}")
                    else:
                        print(f"Arquivo {file} já existe em {output_folder}. Pulando o download.")
        except zipfile.BadZipFile:
            print(f"Erro: O conteúdo de {url} não é um arquivo ZIP válido.")
        except Exception as e:
            print(f"Erro ao extrair arquivos. Erro: {e}")
    else:
        print(f"Erro: Falha na requisição para {url}. Status code: {response.status_code}")

In [108]:
# Iterar sobre as URLs de votação por seção e baixar/extrair os arquivos .csv com "PR" no final
for year, url in votacao_urls.items():
    print(f"Baixando e extraindo dados de votação de {year} para {absolute_output_folder}...")
    download_and_extract(url, absolute_output_folder, filter_extension="PR.csv")

# Iterar sobre as URLs de bens do candidato e baixar/extrair os arquivos .csv com "PR" no final
for year, url in bem_candidato_urls.items():
    print(f"Baixando e extraindo dados de bens do candidato de {year} para {absolute_output_folder}...")
    download_and_extract(url, absolute_output_folder, filter_extension="PR.csv")

# Iterar sobre as URLs dos candidato e baixar/extrair os arquivos .csv com "PR" no final
for year, url in consulta_cand_urls.items():
    print(f"Baixando e extraindo dados dos candidato de {year} para {absolute_output_folder}...")
    download_and_extract(url, absolute_output_folder, filter_extension="PR.csv")

Baixando e extraindo dados de votação de 2020 para /home/crislaine/GitProjects/eleicoes/extracao...
Arquivo votacao_secao_2020_PR.csv já existe em /home/crislaine/GitProjects/eleicoes/extracao. Pulando o download.
Baixando e extraindo dados de votação de 2016 para /home/crislaine/GitProjects/eleicoes/extracao...
Arquivo votacao_secao_2016_PR.csv já existe em /home/crislaine/GitProjects/eleicoes/extracao. Pulando o download.
Baixando e extraindo dados de votação de 2012 para /home/crislaine/GitProjects/eleicoes/extracao...
Arquivo votacao_secao_2012_PR.csv já existe em /home/crislaine/GitProjects/eleicoes/extracao. Pulando o download.
Baixando e extraindo dados de votação de 2008 para /home/crislaine/GitProjects/eleicoes/extracao...
Arquivo votacao_secao_2008_PR.csv já existe em /home/crislaine/GitProjects/eleicoes/extracao. Pulando o download.
Baixando e extraindo dados de votação de 2004 para /home/crislaine/GitProjects/eleicoes/extracao...
Arquivo votacao_secao_2004_PR.csv já existe 

In [115]:
# Função que extrai e processa os dados
def load_and_process_data(path, years, municipio, prefix):
    dfs = {}  # Dicionário para armazenar os DataFrames correspondentes a cada ano

    for year in years:
        # Construção do nome do arquivo a ser carregado
        filename = f'{prefix}_{year}_PR.csv'
        delimiter = ';'
        file_path = os.path.join(absolute_output_folder, filename)

        # Verificar se o arquivo já existe antes de tentar carregar
        if not os.path.exists(file_path):
            print(f"Arquivo para o ano {year} e prefixo {prefix} não encontrado em {path}. Pulando o carregamento.")
            continue

        try:
            # Carregamento do DataFrame a partir do arquivo CSV
            df = pd.read_csv(file_path, encoding='latin-1', on_bad_lines='skip', delimiter=delimiter, dtype={'coluna_problema': str})

            # Filtro para manter apenas os registros relacionados ao município especificado
            if prefix == 'votacao_secao':
                df = df[df['NM_MUNICIPIO'] == municipio]
            elif prefix == 'bem_candidato':
                df = df[df['NM_UE'] == municipio]
            elif prefix == 'consulta_cand':
                df = df[df['NM_UE'] == municipio]
            else:
                print(f"Prefixo {prefix} não reconhecido. Pulando o carregamento.")
                continue

            # Atualização do DataFrame com o nome desejado
            key = f"{year}"
            dfs[key] = df
            print(f"DataFrame carregado com sucesso para o ano {year} e prefixo {prefix}. Nome do DataFrame: dfs_{prefix[0]}['{year}']")
        except Exception as e:
            print(f"Erro ao carregar o DataFrame para o ano {year} e prefixo {prefix}. Erro: {e}")

    return dfs

In [231]:
def format_candidate_names(df, year):
    # Selecionar o DataFrame para o ano desejado e as colunas desejadas
    df_selected = df[year][['SQ_CANDIDATO', 'DS_CARGO', 'NM_CANDIDATO']]
    
    # Remover linhas duplicadas
    df_unique = df_selected.drop_duplicates()
    
    # Criando um dicionário vazio para armazenar os dados formatados
    formatted_dict = {}
    
    # Iterando sobre as linhas do DataFrame
    for index, row in df_unique.iterrows():
        # Dividindo o nome em primeiro e segundo nome
        split_name = row['NM_CANDIDATO'].split(maxsplit=1)
        
        # Limitando o apelido a apenas dois nomes
        apelido = ' '.join(split_name[:2])
        
        # Adicionando ao dicionário formatado
        formatted_dict[row['SQ_CANDIDATO']] = {
            'SQ_CANDIDATO': row['SQ_CANDIDATO'],
            'Nome_Completo': row['NM_CANDIDATO'],
            'Cargo': row['DS_CARGO'],
            'Apelido': apelido,
        }
    
    # Ordenando o dicionário formatado pelo nome completo
    sorted_formatted_dict = dict(sorted(formatted_dict.items(), key=lambda item: item[1]['Nome_Completo']))
        
    # Criando o dicionário dict_candidatos_{year}
    globals()[f"dict_candidatos_{year}"] = sorted_formatted_dict
    
    # Imprimindo a mensagem de conclusão
    print(f"O dicionário dict_candidatos_{year} foi criado com sucesso.")
    
def format_candidate_names_for_years(df, years):
    for year in years:
        format_candidate_names(df, str(year))

In [243]:
def add_apelido_column_for_years(df, years):
    for year in years:
        # Verificar se o ano existe no dicionário
        if year in df:
            dict_candidatos = globals().get(f"dict_candidatos_{year}")

            if dict_candidatos:
                # Verificar se a coluna 'APELIDO' já existe
                if 'APELIDO' not in df[year].columns:
                    # Adicionar a coluna 'Apelido' ao DataFrame do ano
                    df[year]['APELIDO'] = df[year]['SQ_CANDIDATO'].map(lambda sq_candidato: dict_candidatos.get(sq_candidato, {}).get('Apelido'))
                    print(f"A coluna 'Apelido' foi adicionada ao DataFrame {year} com sucesso.")
                else:
                    print(f"A coluna 'Apelido' já existe no DataFrame {year}. Nenhuma ação realizada.")
            else:
                print(f"O dicionário dict_candidatos_{year} não está disponível. Execute a função format_candidate_names antes de adicionar a coluna para o ano {year}.")
        else:
            print(f"O ano {year} não existe no dicionário. Nenhuma ação realizada.")

In [255]:
def process_and_print_bem_candidato(df, year):
    # Verificar se o ano está presente no dicionário
    if year in df:
        # Verificar se as colunas 'APELIDO' e 'VR_BEM_CANDIDATO' existem
        if 'APELIDO' in df[year] and 'VR_BEM_CANDIDATO' in df[year]:
            # Selecionar as colunas desejadas e criar uma cópia do DataFrame
            df_selected = df[year][['APELIDO', 'VR_BEM_CANDIDATO']].copy()

            # Converter a coluna 'VR_BEM_CANDIDATO' para float
            df_selected['VR_BEM_CANDIDATO'] = df_selected['VR_BEM_CANDIDATO'].str.replace(',', '.').str.extract('(\d+\.\d+|\d+)').astype(float)

            # Ordenar o DataFrame pelo 'VR_BEM_CANDIDATO' em ordem decrescente
            df_selected = df_selected.sort_values(by='VR_BEM_CANDIDATO', ascending=False)

            # Formatando a coluna 'VR_BEM_CANDIDATO' para exibição
            df_selected['VR_BEM_CANDIDATO'] = df_selected['VR_BEM_CANDIDATO'].apply(lambda x: f'R$ {x:,.2f}')

            # Imprimir o DataFrame resultante
            print(f"===== Ano {year} =====")
            print(df_selected)
        else:
            print(f"O DataFrame para o ano {year} não contém as colunas 'APELIDO' e/ou 'VR_BEM_CANDIDATO'. Nenhuma ação realizada.")
    else:
        print(f"O ano {year} não está presente no dicionário. Nenhuma ação realizada.")

In [91]:
# Carregando dados do Município

prefix = 'votacao_secao'
absolute_output_folder = absolute_output_folder
years = [2000,2004, 2008, 2012, 2016, 2020]
municipio = 'PIRAQUARA'

dfs_v = load_and_process_data(absolute_output_folder, years, municipio, prefix)

DataFrame carregado com sucesso para o ano 2000 e prefixo votacao_secao. Nome do DataFrame: dfs_v['2000']
DataFrame carregado com sucesso para o ano 2004 e prefixo votacao_secao. Nome do DataFrame: dfs_v['2004']
DataFrame carregado com sucesso para o ano 2008 e prefixo votacao_secao. Nome do DataFrame: dfs_v['2008']
DataFrame carregado com sucesso para o ano 2012 e prefixo votacao_secao. Nome do DataFrame: dfs_v['2012']
DataFrame carregado com sucesso para o ano 2016 e prefixo votacao_secao. Nome do DataFrame: dfs_v['2016']
DataFrame carregado com sucesso para o ano 2020 e prefixo votacao_secao. Nome do DataFrame: dfs_v['2020']


In [92]:
# Carregando dados dos bens de candidatos

prefix = 'bem_candidato'
absolute_output_folder = absolute_output_folder
years = [2008, 2012, 2016, 2020]
municipio = 'PIRAQUARA'

dfs_b = load_and_process_data(absolute_output_folder, years, municipio, prefix)

DataFrame carregado com sucesso para o ano 2008 e prefixo bem_candidato. Nome do DataFrame: dfs_b['2008']
DataFrame carregado com sucesso para o ano 2012 e prefixo bem_candidato. Nome do DataFrame: dfs_b['2012']
DataFrame carregado com sucesso para o ano 2016 e prefixo bem_candidato. Nome do DataFrame: dfs_b['2016']
DataFrame carregado com sucesso para o ano 2020 e prefixo bem_candidato. Nome do DataFrame: dfs_b['2020']


In [127]:
# Carregando dados dos candidatos

prefix = 'consulta_cand'
absolute_output_folder = absolute_output_folder
years = [2000, 2004, 2008, 2012, 2016, 2020]
municipio = 'PIRAQUARA'

dfs_c = load_and_process_data(absolute_output_folder, years, municipio, prefix)

DataFrame carregado com sucesso para o ano 2000 e prefixo consulta_cand. Nome do DataFrame: dfs_c['2000']
DataFrame carregado com sucesso para o ano 2004 e prefixo consulta_cand. Nome do DataFrame: dfs_c['2004']
DataFrame carregado com sucesso para o ano 2008 e prefixo consulta_cand. Nome do DataFrame: dfs_c['2008']
DataFrame carregado com sucesso para o ano 2012 e prefixo consulta_cand. Nome do DataFrame: dfs_c['2012']
DataFrame carregado com sucesso para o ano 2016 e prefixo consulta_cand. Nome do DataFrame: dfs_c['2016']
DataFrame carregado com sucesso para o ano 2020 e prefixo consulta_cand. Nome do DataFrame: dfs_c['2020']


In [241]:
# Carregando dicionários dos candidatos

years = [2000, 2004, 2008, 2012, 2016, 2020]

format_candidate_names_for_years(dfs_c, years)

O dicionário dict_candidatos_2000 foi criado com sucesso.
O dicionário dict_candidatos_2004 foi criado com sucesso.
O dicionário dict_candidatos_2008 foi criado com sucesso.
O dicionário dict_candidatos_2012 foi criado com sucesso.
O dicionário dict_candidatos_2016 foi criado com sucesso.
O dicionário dict_candidatos_2020 foi criado com sucesso.


In [247]:
# Adicionar a coluna 'Apelido' aos Dataframes

years = [2000, 2004, 2008, 2012, 2016, 2020]

add_apelido_column_for_years(dfs_b, years)

O ano 2000 não existe no dicionário. Nenhuma ação realizada.
O ano 2004 não existe no dicionário. Nenhuma ação realizada.
O ano 2008 não existe no dicionário. Nenhuma ação realizada.
O ano 2012 não existe no dicionário. Nenhuma ação realizada.
O ano 2016 não existe no dicionário. Nenhuma ação realizada.
O ano 2020 não existe no dicionário. Nenhuma ação realizada.


In [256]:
# Verificando bens dos candidatos

years = [2000, 2004, 2008, 2012, 2016, 2020]

for year in years:
    process_and_print_bem_candidato(dfs_b, str(year))

O ano 2000 não está presente no dicionário. Nenhuma ação realizada.
O ano 2004 não está presente no dicionário. Nenhuma ação realizada.
O DataFrame para o ano 2008 não contém as colunas 'APELIDO' e/ou 'VR_BEM_CANDIDATO'. Nenhuma ação realizada.
===== Ano 2012 =====
                                       APELIDO VR_BEM_CANDIDATO
71427                GIUSEPPE GOMEZ MIGLIORINI    R$ 827,000.00
55368                            MARILDO GOMES    R$ 500,000.00
55359                   HELTON CHAGAS DA SILVA    R$ 450,000.00
27568                       ARMANDO NEME FILHO    R$ 400,000.00
71406                      ALCEU BORBA RESENDE    R$ 390,000.00
55330                      SIDNEI CESAR MAMEDE    R$ 350,000.00
50042                    JOSÉ GENTIL DE CASTRO    R$ 350,000.00
71448                         JOSE CARLOS NATO    R$ 300,000.00
67676                          ADEMIR PICANCIO    R$ 250,000.00
71430                GIUSEPPE GOMEZ MIGLIORINI    R$ 250,000.00
50501                         

In [ ]:
# Função para gerar perfil dos dados

def generate_profile_report(dfs, year, cargo):
    # Obtenção do DataFrame correspondente ao ano
    df = df_final.get(year)

    # Verificação se o DataFrame existe e não está vazio para o cargo específico
    if df is not None and not df[(df["DS_CARGO"] == cargo)].empty:
        # Filtragem do DataFrame para o cargo específico
        df_filtered = df[df["DS_CARGO"].str.lower() == cargo.lower()]
        # Agrupamento por candidato e soma dos votos
        df_grouped = df_filtered.groupby("NM_VOTAVEL")["QT_VOTOS"].sum().reset_index()
        # Ordenação por quantidade de votos
        df_sorted = df_grouped.sort_values(by="QT_VOTOS", ascending=False)
        # Geração do perfil de dados
        profile = ProfileReport(df_sorted, title=f"Piraquara - {cargo} {year}")
        # Salvar perfil em um arquivo HTML
        profile.to_file(f"Piraquara - {cargo} {year}.html")
    else:
        print(f"No data found for the year {year} and cargo {cargo}.")


# Exemplo de uso da função para gerar perfis de dados
selected_year = 2020
generate_profile_report(df_final, selected_year, "Prefeito")
generate_profile_report(df_final, selected_year, "Vereador")

In [ ]:
# Função que plota os candidatos em um gráfico de barras

def plot_top_candidates(df, cargo, top_n, include_branco_nulo=True, include_partidos=True):
    df_filtered = df[df['DS_CARGO'].str.lower() == cargo.lower()]

    if not include_branco_nulo:
        df_filtered = df_filtered[~df_filtered['NM_VOTAVEL'].isin(['VOTO BRANCO', 'VOTO NULO'])]

    if not include_partidos:
        df_filtered = df_filtered[
            ~((df_filtered['NM_VOTAVEL'].str.lower().str.contains('partido', na=False)) |
              (df_filtered['NM_VOTAVEL'].str.len() <= 5))
        ]

    df_grouped = df_filtered.groupby('NM_VOTAVEL')['QT_VOTOS'].sum().reset_index()
    df_sorted = df_grouped.sort_values(by='QT_VOTOS', ascending=False)
    top_candidates = df_sorted.head(top_n)

    plt.figure(figsize=(12, 8))
    bars = plt.bar(top_candidates['NM_VOTAVEL'], top_candidates['QT_VOTOS'], color='skyblue')
    plt.title(f'Top {top_n} - Votos para {cargo} em Piraquara')
    plt.xlabel('Candidatos')
    plt.ylabel('Total Votos')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 100, round(yval), ha='center', va='bottom', color='black',
                 fontsize=8)

    plt.show()


# Função que plota os votos do partido

def plot_votes_by_partido(df, selected_year):
    df_filtered = df[df['ANO_ELEICAO'] == selected_year]
    df_grouped = df_filtered.groupby('NM_VOTAVEL')['QT_VOTOS'].sum().reset_index()

    df_grouped = df_grouped[
        (df_grouped['NM_VOTAVEL'].str.lower().str.startswith('partido', na=False)) |
        (df_grouped['NM_VOTAVEL'].str.len() <= 5)
    ]

    plt.figure(figsize=(12, 8))
    bars = plt.bar(df_grouped['NM_VOTAVEL'], df_grouped['QT_VOTOS'], color='skyblue')
    plt.title(f'Votos por Partido - {selected_year}')
    plt.xlabel('Partidos')
    plt.ylabel('Total Votos')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 100, round(yval), ha='center', va='bottom', color='black',
                 fontsize=8)

    plt.show()


# Função que plota a evolução dos candidatos

def plot_timeline_all_years(dfs, selected_candidato):
    # Convertendo o nome do candidato para minúsculas e removendo acentos
    selected_candidato = unidecode.unidecode(selected_candidato.lower())

    # Lista para armazenar os resultados de cada DataFrame
    resultados = []

    # Iterando sobre os DataFrames
    for year, df in dfs.items():
        # Filtragem dos registros com base no nome do candidato
        mask = df['NM_VOTAVEL'].str.lower().str.contains(selected_candidato)
        df_filtered = df[mask].copy()  # Criando uma cópia usando o método copy

        # Verificando se há registros correspondentes
        if not df_filtered.empty:
            # Preenchendo valores ausentes com zero na coluna 'QT_VOTOS' usando .loc
            df_filtered.loc[:, 'QT_VOTOS'] = df_filtered['QT_VOTOS'].fillna(0).astype(int)

            # Iterando sobre os candidatos encontrados
            for _, row in df_filtered.iterrows():
                # Somando o total de votos para cada candidato nos anos
                total_votos = row['QT_VOTOS']
                nome_candidato = row['NM_VOTAVEL']

                # Adicionando o resultado à lista
                resultados.append((year, total_votos, nome_candidato))

    if not resultados:
        print(f"Nenhum registro encontrado para o candidato: {selected_candidato}")
        return

    # Criando DataFrame para facilitar o plot
    df_plot = pd.DataFrame(resultados, columns=['Ano', 'Votos', 'Nome'])

    # Agrupando por Nome e Ano e somando os votos
    df_plot = df_plot.groupby(['Nome', 'Ano'])['Votos'].sum().reset_index()

    # Plotando o gráfico de barras
    plt.figure(figsize=(12, 8))
    ax = sns.barplot(x='Ano', y='Votos', hue='Nome', data=df_plot, palette='viridis')
    plt.title(f'Total de Votos ao Longo dos Anos para Candidatos - {selected_candidato}')
    plt.xlabel('Ano da Eleição')
    plt.ylabel('Total Votos')
    plt.tight_layout()
    plt.grid(True)

    # Adição de rótulos com os totais de votos sobre as barras
    for p in ax.patches:
        if pd.notna(p.get_height()):  # Verificando se o valor não é NaN
            ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center', fontsize=8, color='black', xytext=(0, 10),
                        textcoords='offset points')

    plt.show()

In [ ]:
# Ano da Eleição
selected_year = 2020

# Cargo (Vereador ou Prefeito)
selected_cargo = 'Vereador'

# Quantidade de Candidatos
top_n_candidates = 25

# Incluir Votos Brancos e Nulos? 
include_branco_nulo = False

# Incluir votos no Partido?
include_partidos = False

# Chamada da função
plot_top_candidates(df_final[selected_year], selected_cargo, top_n_candidates, include_branco_nulo, include_partidos)
plot_votes_by_partido(df_final[selected_year], selected_year)

In [ ]:
# Nome do Candidato
selected_candidato = 'Wagner'

plot_timeline_all_years(df_final, selected_candidato)